In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from src.plotting_tools.SysHist import make_hist, make_sys_hist, SysHist
from src.data_tools.get_data import get_data
from src.plotting_tools.Bins import Bins, bins, binning_type
from src.plotting_tools.cms_format import cms_format_fig, cms_style
from src.plotting_tools.colors import color_fader

In [ ]:
cms_style()

In [ ]:
era = '2016'
output_dir = 'assets_nov_2022'
df, lumi = get_data(era, output_dir, stitch_dy=1)

In [ ]:
from src.assets.regions import nJets, regions, region_and_label
from src.assets.file_groups import bck_dict, bck_list, bck_colors


In [ ]:
from src.plotting_tools.draw_stack_plot_hists import draw_bckground, draw_signals, draw_data, draw_stackplot

In [ ]:
from src.plotting_tools.latexAssets import HTLT, RelMET

In [ ]:
draw_inline = True
if draw_inline:
    plt.ion()
else:
    plt.ioff()

In [ ]:
#HTLT relmet plt
# select_level=.5 ensures that its pre bff seleciton cuts
reg = 'SR1'
feature = 'HTLT'
fig, ax = plt.subplots(figsize=(12,12))
draw_stackplot(ax, df, feature, reg,era, lumi, draw_bck_sys=0, draw_sig_sys=0, top=1e5,  
               bin_edges=np.linspace(-800,800,100),
               make_density=0,
             select_level=.5,
              xlabel=HTLT)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, era, reg))

reg = 'SR1'
feature = 'RelMET'
fig, ax = plt.subplots(figsize=(12,12))
draw_stackplot(ax, df, feature, reg,era, lumi, draw_bck_sys=0, draw_sig_sys=0, top=1e5, 
               bin_edges=np.linspace(0, 2,100),
               make_density=0,
              select_level=.5,
              xlabel= RelMET)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, era, reg))



In [ ]:
#HTLT relmet plt
# select_level=.5 ensures that its pre bff seleciton cuts
reg = 'SR2'
feature = 'HTLT'
fig, ax = plt.subplots(figsize=(12,12))
draw_stackplot(ax, df, feature, reg,era, lumi, draw_bck_sys=0, draw_sig_sys=0, top=1e5, 
            make_density=0,
            bin_edges=np.linspace(-800,800,100),
            select_level=.5,
              xlabel=HTLT)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, era, reg))

reg = 'SR2'
feature = 'RelMET'
fig, ax = plt.subplots(figsize=(12,12))
draw_stackplot(ax, df, feature, reg,era, lumi, draw_bck_sys=0, draw_sig_sys=0, top=1e5, 
            make_density=0,
            bin_edges=np.linspace(0, 2,100),
            select_level=.5,
              xlabel= RelMET)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, era, reg))



In [ ]:
background_df = df[df.type.str.contains('bck')]
_bhist = make_sys_hist(background_df, feature, reg)

In [ ]:
dy_sel = (df.category.str.contains('DY')) & (df.category != 'DYLL')
dy_types = df[dy_sel].category.unique()

In [ ]:
dy_types

In [ ]:
feature, reg = "DiLepMass", "CR10"
for name in df.name.unique():
    ndf = df[df.name==name]
    _bhist = make_sys_hist(ndf, feature, reg)
    print(name)
    print(_bhist.sys_summary())

In [ ]:
reg = 'CR10'
feature = 'DiLepMass'
fig, ax = plt.subplots(figsize=(12,12))
draw_stackplot(ax, df, feature, reg,era, lumi, draw_bck_sys=1)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, era, reg))


In [ ]:
for reg in regions:
    fig, ax = plt.subplots(figsize=(12,12))
    draw_stackplot(ax, df, feature, reg,era, lumi)
    print(reg)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, era, reg))


In [ ]:
##
## needs to be run after abcd fits 
##

In [ ]:
file = '{}/fits/limit_setting/abcd_dict_{}.pkl'.format(output_dir, era)
import pickle
import os
import sys
if not os.path.exists(file): sys.exit()

In [ ]:
with open(file, 'rb') as f:
    abcd_fit_dict = pickle.load(f)

In [ ]:
from src.plotting_tools.make_abcd_hist import make_abcd_hist

In [ ]:
for reg, abdc_arrays in abcd_fit_dict.items():
    fig, ax = plt.subplots(figsize=(12,12))
    draw_stackplot(ax, df, feature, reg,era, lumi,draw_bck_sys=True,error_scale=0)
    #abcd hist
    abcd_hist = make_abcd_hist(abdc_arrays)
    abcd_hist.make_density_hist().draw(ax, label='ABCD Pred.', color='green')
    ax.legend(ncol=2)
    fig.savefig('{}/stack_plots/{}_{}_{}_abcd.pdf'.format(output_dir, feature, era, reg))


In [ ]:
##
## needs to be run after limts 
##

In [ ]:
limit_df = pd.read_csv('{}/combine_data/{}_count_limits.csv'.format(outdir, era))

In [ ]:
tldf = limit_df[limit_df.dbs==0.1]
reg1 = tldf[tldf.nJets=='SR1']
reg1 = reg1.sort_values('mass')
count = reg1['50.0']
mass = reg1['mass']

In [ ]:
def interpolate_limit(nJets, mass):
    tldf = limit_df[limit_df.dbs==0.1]
    reg1 = tldf[tldf.nJets==nJets]
    reg1 = reg1.sort_values('mass')
    counts = reg1['50.0']
    masses = reg1['mass']
    return np.interp(mass, masses, counts)

In [ ]:
sdf = df[df.type=='sig']

In [ ]:
#reweight signals to the expected limits
Weight_columns = df.filter(regex='^Weight').columns
for mass in sdf['mass'].unique():
    for dbs in sdf['dbs'].unique():
        for reg in ['SR1', 'SR2']:
            count_limit = interpolate_limit(reg, mass)
            reg_sel = df.filter(regex='{}'.format(reg)).sum(axis=1) > 0
            sample_select = (df.dbs==dbs) & (df.mass==mass)
            nEvents = df[sample_select & (df[reg+'_jet_nom_muon_corrected_pt_ele_pt']==1)].Weight.sum()
            reweight = count_limit/nEvents
            df.loc[sample_select & reg_sel, Weight_columns] *= reweight


In [ ]:
feature = 'DiLepMass'
for reg, abdc_arrays in abcd_fit_dict.items():
    fig, ax = plt.subplots(figsize=(12,12))
    draw_stackplot(ax, df, feature, reg,era, lumi,draw_bck_sys=True,error_scale=0)
    #abcd hist
    abcd_hist = make_abcd_hist(abdc_arrays)
    abcd_hist.make_density_hist().draw(ax, label='ABCD Pred.', color='green')
    ax.legend(ncol=2)
    fig.savefig('{}/stack_plots/{}_{}_{}_abcd_limits.pdf'.format(output_dir, feature, era, reg))


In [ ]:
feature = 'DiLepMass'
bin_edges=np.linspace(110, 200, (200-110+1))
for reg, abdc_arrays in abcd_fit_dict.items():
    fig, ax = plt.subplots(figsize=(12,12))
    draw_stackplot(ax, df, feature, reg,era, lumi,draw_bck_sys=True,error_scale=0, yscale='linear')
    #abcd hist
    abcd_hist = make_abcd_hist(abdc_arrays)
    abcd_hist.make_density_hist().draw(ax, label='ABCD Pred.', color='green')
    ax.legend(ncol=2)
    ax.set_xlim(110,200)
    ax.set_ylim(0,150)
    fig.savefig('{}/stack_plots/{}_{}_{}_abcd_limits_200_max.pdf'.format(output_dir, feature, era, reg))
